In [ ]:
import pandas as pd

def read_csv_with_multiple_encodings(file_path, encodings=['utf-8', 'latin1', 'ISO-8859-1', 'cp1252']):
    for encoding in encodings:
        try:
            df = pd.read_csv(file_path, encoding=encoding)
            print(f"File read successfully with encoding: {encoding}")
            return df
        except UnicodeDecodeError:
            print(f"Failed to read file with encoding: {encoding}")
    raise ValueError("Unable to read the file with the provided encodings")

file_path = "E://files_pandas//datos_abiertos//IRS_localidad_2020.csv"
df_irs = read_csv_with_multiple_encodings(file_path)


DATA FRAME ANALFABETISMO

In [ ]:
df_irs_hgo = df_irs[df_irs['ent'] == 'Hidalgo']
columnas = ['cve_loc','mun','loc','i_analf']
df_analf = df_irs_hgo[ columnas]
df_analf = df_analf.sort_values(by='i_analf', ascending=False)
df_geoloc = pd.read_csv('E://files_pandas//sisver//AGEEML_202410311056515.csv') # csv con las coordenadas de las localidades 
df_analf = df_analf.merge(df_geoloc[['NOM_LOC', 'LAT_DECIMAL', 'LON_DECIMAL']], 
                          left_on='loc', right_on='NOM_LOC', how='left')# merge de los dos df para obtener las coordenadas 

df_analf.rename(columns={'LAT_DECIMAL': 'lat', 'LON_DECIMAL': 'lon'}, inplace=True)#renombrar columnas 
df_analf = df_analf.dropna(subset=['lat', 'lon'])#quitar los missing values para poder mapear


CONTORNO MAPA DEL ESTADO

In [ ]:
import geopandas as gpd
import folium
import pandas as pd
from folium.plugins import HeatMap
# Ruta al archivo SHP
shapefile_path = 'E://mapas//13_hidalgo//conjunto_de_datos//13ent.shp'

# Leer el archivo SHP
shpg = gpd.read_file(shapefile_path)

# Filtrar por la columna NOMGEO que contiene 'HIDALGO'
shpg_hidalgo = shpg[shpg['NOMGEO'].str.contains('Hidalgo', case=False)]

# Crear el mapa base centrado en Hidalgo
m = folium.Map(location=[20.0911, -98.7624], zoom_start=8)  # Coordenadas aproximadas de Hidalgo

def estilo_poligono(feature): 
    return { 
        'color': 'black', # Color de borde 
        'weight': 2, # Grosor del borde 
        'fillOpacity': 0.0 # Transparencia del relleno 
        }

folium.GeoJson(shpg_hidalgo, style_function=estilo_poligono).add_to(m)

heat_data = [[row['lat'], row['lon'], row['i_analf']] for index, row in df_analf.iterrows()]

# Agregar el mapa de calor al mapa base
HeatMap(heat_data).add_to(m)
m

m.save('E://files_pandas//datos_abiertos//mapas//mapa_calor_analfabetismo_hidalgo.html')

Obteniendo promedio de indice de analfabetizmo por municipio, agrupar y generar columna calculada 

In [ ]:
df_mpio_anf = df_analf[['mun','i_analf']]
promedio_analf = df_mpio_anf.groupby('mun')['i_analf'].mean().reset_index()
promedio_analf.rename(columns={'i_analf': 'promedio_i_analf'}, inplace=True)
promedio_analf= promedio_analf.sort_values(by='promedio_i_analf', ascending=False)
promedio_analf['promedio_i_analf'] = promedio_analf['promedio_i_analf'].round(2)

In [ ]:
import geopandas as gpd
import folium

# Ruta al archivo SHP o GeoJSON
try:
    gdf = gpd.read_file("E://files_pandas//sisver//muni_2018gw//muni_2018gw.shp", encoding='utf-8')
except UnicodeDecodeError:
    gdf = gpd.read_file("E://files_pandas//sisver//muni_2018gw//muni_2018gw.shp", encoding='latin-1')

# Leer el archivo SHP

gdf = gdf[gdf['NOM_ENT'] == 'Hidalgo']
# Crear el mapa base centrado en Hidalgo
m = folium.Map(location=[20.0911, -98.7624], zoom_start=8)  # Coordenadas aproximadas de Hidalgo

# Crear un GeoDataFrame con los datos de promedio_analf
gdf = gdf.merge(promedio_analf, left_on='NOM_MUN', right_on='mun')

# Crear el Choropleth
folium.Choropleth(
    geo_data=gdf,
    name='choropleth',
    data=gdf,
    columns=['mun', 'promedio_i_analf'],
    key_on='feature.properties.NOM_MUN',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Promedio de Índice de Analfabetismo'
).add_to(m)

# Agregar capas de control
folium.LayerControl().add_to(m)

# Guardar el mapa en un archivo HTML
m.save('E://files_pandas//sisver//mapa_analfabetismo_hidalgo.html')

# Mostrar el mapa
m

e:\Repositorios\open_data\datos_abiertos\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: Invalid range specified

  return ogr_read(


DataSourceError: CVE_ENT CVE_MUN CVEGEO         NOM_ENT          AREA   PERIMETER  \0         01     001  01001  Aguascalientes  116635.92455  1166.35925   1         01     002  01002  Aguascalientes   54337.42654   543.37427   2         01     003  01003  Aguascalientes   92335.78320   923.35783   3         01     004  01004  Aguascalientes   12838.35453   128.38355   4         01     005  01005  Aguascalientes   49989.50926   499.89509   ...      ...     ...    ...             ...           ...         ...   2458      32     054  32054       Zacatecas   37142.59872   371.42599   2459      32     055  32055       Zacatecas  215650.56021  2156.50560   2460      32     056  32056       Zacatecas   43678.57090   436.78571   2461      32     057  32057       Zacatecas   21844.70068   218.44701   2462      32     058  32058       Zacatecas   27566.66479   275.66665                       NOM_MUN  COV_  COV_ID  \0            Aguascalientes     0       1   1                  Asientos     1   

In [ ]:
import geopandas as gpd

# Ruta al archivo SHP
try:
    gdf = gpd.read_file("E://files_pandas//sisver//muni_2018gw//muni_2018gw.shp", encoding='utf-8')
except UnicodeDecodeError:
    gdf = gpd.read_file("E://files_pandas//sisver//muni_2018gw//muni_2018gw.shp", encoding='latin-1')

# Leer el archivo SHP
gdf = gpd.read_file(shapefile_path)
gdf = gdf[gdf['NOM_ENT'] == 'Hidalgo']
# Mostrar las primeras filas del GeoDataFrame
print(gdf.head())


    CVE_ENT CVE_MUN CVEGEO  NOM_ENT         AREA  PERIMETER  \
452      13     001  13001  Hidalgo  23978.12399  239.78124   
453      13     002  13002  Hidalgo  23708.80399  237.08804   
454      13     003  13003  Hidalgo  26969.47989  269.69480   
455      13     004  13004  Hidalgo  11907.74028  119.07740   
456      13     005  13005  Hidalgo  23722.94946  237.22949   

                     NOM_MUN  COV_  COV_ID  \
452                 AcatlÃ¡n   452     453   
453            AcaxochitlÃ¡n   453     454   
454                  Actopan   454     455   
455  Agua Blanca de Iturbide   455     456   
456                  Ajacuba   456     457   

                                              geometry  
452  POLYGON ((-98.45855 20.33365, -98.45834 20.333...  
453  POLYGON ((-98.14701 20.22061, -98.14724 20.220...  
454  POLYGON ((-98.81566 20.38951, -98.81594 20.388...  
455  POLYGON ((-98.3341 20.32889, -98.33468 20.3279...  
456  POLYGON ((-99.04028 20.18884, -99.04026 20.188...  
